# Prerequisites

Download and install required packages for this notebook:
* Captum: Model Interpretability for PyTorch
* Optuna: A Hyperparameter Optimization Framework
* Kaleido: Generating Static Images for Optuna Plots

In [ ]:
%pip install captum
%pip install optuna
%pip install kaleido

# Library Imports & Constants

This section imports all the necessary libraries and defines constants used throughout the notebook.

In [ ]:
import json
import random
from typing import Optional

import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
from captum.attr import IntegratedGradients
from optuna.trial import TrialState
from optuna.visualization import (
    plot_contour,
    plot_intermediate_values,
    plot_optimization_history,
    plot_param_importances,
    plot_parallel_coordinate,
    plot_slice
)
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

TRAINING_DATA_FILEPATH = 'train.csv'
VAL_SPLIT = 10
N_COMPANIES = 442
N_TRIALS = 50
TRIAL_TIMEOUT = None
BEST_PARAMS_FILEPATH = 'best_params.json'
BEST_MODEL_FILEPATH = 'best_model.pth'

# Exploratory Data Analysis

This section performs an initial analysis of the dataset by visualisaing trends of the stock percentage changes for a few random companies over time.

In [ ]:
data = pd.read_csv(TRAINING_DATA_FILEPATH, index_col='ID')
data.head()

In [ ]:
N_SAMPLE_COMPANIES = 5
LAST_N_DAYS = 365
random_samples = random.sample(data.index.tolist(), N_SAMPLE_COMPANIES)
plt.figure(figsize=(10, 6))


for company in random_samples:
    plt.plot(data.columns[-LAST_N_DAYS:], data.loc[company, data.columns[-LAST_N_DAYS:]], label=company)

plt.xlabel('Date')
plt.ylabel('Daily Percentage Movement')
plt.title(f'Time Series of Stock Price Changes for {N_SAMPLE_COMPANIES} Random Companies over {LAST_N_DAYS} Days')
plt.xticks([])
plt.legend()
plt.tight_layout()
plt.savefig('time_series_trends.png')
plt.show()

Plotting the seasonality, shows that most companies go through a series of ups and downs. However, here for example `company_195` witnessed a significant jump of ~24% in a single day. Additionally, outlier events like Covid affected the values of stock market changes drastically. This exploratory data analysis suggested that perhaps if normalisation was performed for every company, instead of the entire dataset, the effect of outliers may be reduced.

![](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/time_series_trends.png)

# Custom Dataset

This section defines the `StockDataset` class, an implementation of PyTorch's `Dataset` to load custom data as done in Lab 5.
It takes in several parameters:
* `csv_file_path`: The file path for `train.csv`.
* `in_seq_length`: The number of lookback days for the time series prediction.
* `is_train`: A flag to indicate if the loader is being used for training or validation.
* `train_val_split`: A numeric value indicating how much of the dataset should be used for validation.
* `scalers`: In case of validation, the same `StandardScaler` instances from training are re-used to prevent data leakage.

As mentioned before, this custom implementation performs a train/validation split and performs normalisation at a per company level.

In [ ]:
class StockDataset(Dataset):
    def __init__(self,
                 csv_file_path: str,
                 in_seq_length: int,  # number of previous days to consider
                 is_train: bool,
                 train_val_split: int,  # number of days for validation set
                 scalers: Optional[list] = None):
        # process raw data
        self.df = pd.read_csv(csv_file_path) # (442, 3022)
        self.df = self.df.drop(columns=['ID'])  # (442, 3021)
        # define parameters
        self.in_seq_length = in_seq_length

        # train / val split
        self.split_idx = self.df.shape[1] - train_val_split
        if is_train:
            self.data = self.df.iloc[:, :self.split_idx]
        else:
            self.data = self.df.iloc[:, (self.split_idx - in_seq_length):]

        # per company normalisation
        self.normalized_data = np.zeros_like(self.data.values)
        if scalers is None:
            self.scalers = []
            for idx in range(self.df.shape[0]):
                scaler = StandardScaler()
                company_data = self.data.iloc[idx].values.reshape(-1, 1)
                self.normalized_data[idx] = scaler.fit_transform(company_data).flatten()
                self.scalers.append(scaler)
        else:
            self.scalers = scalers
            for idx in range(self.df.shape[0]):
                company_data = self.data.iloc[idx].values.reshape(-1, 1)
                self.normalized_data[idx] = self.scalers[idx].transform(company_data).flatten()

    def __len__(self):
        # number of possible sequences
        return self.normalized_data.shape[1] - self.in_seq_length

    def __getitem__(self, idx: int) -> dict:
        # get sequence window
        X = self.normalized_data[:, idx:idx + self.in_seq_length]  # (442, seq_length)
        # get target values (next day for all companies)
        y = self.normalized_data[:, idx + self.in_seq_length]  # (442,)

        X = torch.FloatTensor(X.T)  # for LSTM (seq_length, 442)
        y = torch.FloatTensor(y)
        return {'X': X, 'y': y}

# LSTM Model

This section defines the `StockLSTM` class, a PyTorch implementation of an LSTM model similar to Lab 7.
The model initialises the LSTM hidden and cell states to zero, passes through the LSTM and applies a final linear layer to predict the next day's stock movements for all 442 companies.

In [ ]:
class StockLSTM(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, num_layers: int, dropout: float):
        super(StockLSTM, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x shape: (batch_size, seq_length, input_dim)
        # tnitialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        # pass through LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: (batch_size, seq_length, hidden_dim)
        # take the final time-step
        out = out[:, -1, :]  # shape: (batch_size, hidden_dim)
        # apply dropout and final linear layer
        out = self.dropout(out)
        out = self.fc(out)  # shape: (batch_size, input_dim)
        return out

# Hyperparameter Tuning using Optuna

This section uses Optuna to perform hyperparameter optimization for the LSTM model similar to Lab 4. The objective function to optimise here is the mean squared error over the validation dataset. The following hyperparameters are defined:
* `sequence_length`: The number of lookback days for the LSTM model.
* `hidden_dim`: The dimensions of the latent layers.
* `num_layers`: The number of layers in the LSTM.
* `dropout`: The dropout rate to prevent overfitting.
* `learning_rate`: The learning rate for the optimizer.
* `batch_size`: The batch size for training and validation.
* `num_epochs`: The number of epochs to train the model.
* `optimizer`: The optimisation algorithm to use.

In [ ]:
def objective(trial: optuna.Trial):
    PARAMETERS = {
        'sequence_length': trial.suggest_int('sequence_length', 3, 150),
        'hidden_dim': trial.suggest_int('hidden_dim', 32, 256),
        'num_layers': trial.suggest_int('num_layers', 1, 3),
        'dropout': trial.suggest_float('dropout', 0.0, 0.5),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [8, 16, 32, 64, 128]),
        'num_epochs': trial.suggest_int('num_epochs', 30, 100),
        'optimizer': trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
    }
    # create datasets
    train = StockDataset(
        csv_file_path=TRAINING_DATA_FILEPATH,
        in_seq_length=PARAMETERS['sequence_length'],
        is_train=True,
        train_val_split=VAL_SPLIT)
    validation = StockDataset(
        csv_file_path=TRAINING_DATA_FILEPATH,
        in_seq_length=PARAMETERS['sequence_length'],
        is_train=False,
        train_val_split=VAL_SPLIT,
        scalers=train.scalers)
    # create data loaders
    train_loader = DataLoader(
        train,
        batch_size=PARAMETERS['batch_size'],
        shuffle=True)
    val_loader = DataLoader(
        validation,
        batch_size=PARAMETERS['batch_size'],
        shuffle=False)

    # create model
    model = StockLSTM(
        input_dim=N_COMPANIES,
        hidden_dim=PARAMETERS['hidden_dim'],
        num_layers=PARAMETERS['num_layers'],
        dropout=PARAMETERS['dropout']
    ).to(DEVICE)
    # loss and optimizer
    criterion = nn.MSELoss()
    optimizer = getattr(optim, PARAMETERS['optimizer'])(
        model.parameters(),
        lr=PARAMETERS['learning_rate']
    )
    # learning rate scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5,
    )

    # training loop
    train_losses, val_losses = [], []
    best_val_loss = float('inf')
    for epoch in range(PARAMETERS['num_epochs']):
        # training phase
        model.train()
        total_train_loss = 0
        for batch in train_loader:
            X = batch['X'].to(DEVICE)  # (batch size, sequence_length, 442)
            y = batch['y'].to(DEVICE)  # (batch size, 442)
            optimizer.zero_grad()
            outputs = model(X)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        avg_train_loss = total_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # validation phase
        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                X = batch['X'].to(DEVICE)
                y = batch['y'].to(DEVICE)
                outputs = model(X)
                loss = criterion(outputs, y)
                total_val_loss += loss.item()
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        # update learning rate scheduler
        scheduler.step(avg_val_loss)

        trial.report(avg_val_loss, epoch)
        # handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss

    return best_val_loss

In [ ]:
study = optuna.create_study(study_name='Stock LSTM Hyperparameter Tuning', direction='minimize')
study.optimize(objective, n_trials=N_TRIALS, timeout=TRIAL_TIMEOUT)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print(" Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)

with open(BEST_PARAMS_FILEPATH, 'w') as outfile:
    json.dump(trial.params, outfile)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

<details open>
  <summary>Stock LSTM Hyperparameter Tuning Study Trials</summary>
  
```  
[I 2025-03-20 20:31:45,072] A new study created in memory with name: Stock LSTM Hyperparameter Tuning
[I 2025-03-20 20:32:58,519] Trial 0 finished with value: 1.0372108221054077 and parameters: {'sequence_length': 147, 'hidden_dim': 234, 'num_layers': 1, 'dropout': 0.33930960583094977, 'learning_rate': 0.00021421665569423075, 'batch_size': 64, 'num_epochs': 60, 'optimizer': 'RMSprop'}. Best is trial 0 with value: 1.0372108221054077.
[I 2025-03-20 20:33:50,232] Trial 1 finished with value: 0.737079918384552 and parameters: {'sequence_length': 87, 'hidden_dim': 153, 'num_layers': 2, 'dropout': 0.4755953347734666, 'learning_rate': 0.0017851621886235617, 'batch_size': 16, 'num_epochs': 34, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:36:41,637] Trial 2 finished with value: 0.7578898668289185 and parameters: {'sequence_length': 39, 'hidden_dim': 136, 'num_layers': 3, 'dropout': 0.009576055372586267, 'learning_rate': 0.0001312467662147458, 'batch_size': 8, 'num_epochs': 77, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:37:34,095] Trial 3 finished with value: 1.0775812864303589 and parameters: {'sequence_length': 42, 'hidden_dim': 195, 'num_layers': 3, 'dropout': 0.4761042233260503, 'learning_rate': 0.00890964708789742, 'batch_size': 64, 'num_epochs': 80, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:37:49,886] Trial 4 finished with value: 1.087605595588684 and parameters: {'sequence_length': 23, 'hidden_dim': 124, 'num_layers': 1, 'dropout': 0.06866701681814286, 'learning_rate': 0.00011499798085947112, 'batch_size': 128, 'num_epochs': 61, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:37:51,422] Trial 5 pruned. 
[I 2025-03-20 20:37:53,880] Trial 6 pruned. 
[I 2025-03-20 20:39:41,367] Trial 7 finished with value: 0.9135304093360901 and parameters: {'sequence_length': 103, 'hidden_dim': 56, 'num_layers': 3, 'dropout': 0.46685690087632264, 'learning_rate': 0.005107658354735002, 'batch_size': 8, 'num_epochs': 65, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:39:43,281] Trial 8 pruned. 
[I 2025-03-20 20:39:44,910] Trial 9 pruned. 
[I 2025-03-20 20:39:49,927] Trial 10 pruned. 
[I 2025-03-20 20:43:06,551] Trial 11 finished with value: 0.7578939199447632 and parameters: {'sequence_length': 75, 'hidden_dim': 151, 'num_layers': 2, 'dropout': 0.20755764055341164, 'learning_rate': 0.0005332498443319924, 'batch_size': 8, 'num_epochs': 84, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:43:09,183] Trial 12 pruned. 
[I 2025-03-20 20:44:18,072] Trial 13 finished with value: 0.8719300031661987 and parameters: {'sequence_length': 50, 'hidden_dim': 98, 'num_layers': 2, 'dropout': 0.0033603142132102156, 'learning_rate': 0.0006256986847047751, 'batch_size': 8, 'num_epochs': 50, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:44:24,776] Trial 14 pruned. 
[I 2025-03-20 20:44:26,131] Trial 15 pruned. 
[I 2025-03-20 20:46:49,218] Trial 16 finished with value: 0.8159942030906677 and parameters: {'sequence_length': 64, 'hidden_dim': 237, 'num_layers': 2, 'dropout': 0.117299654429115, 'learning_rate': 0.0012834787561936448, 'batch_size': 16, 'num_epochs': 91, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:46:51,984] Trial 17 pruned. 
[I 2025-03-20 20:46:54,608] Trial 18 pruned. 
[I 2025-03-20 20:46:56,151] Trial 19 pruned. 
[I 2025-03-20 20:46:58,613] Trial 20 pruned. 
[I 2025-03-20 20:47:06,870] Trial 21 pruned. 
[I 2025-03-20 20:47:10,176] Trial 22 pruned. 
[I 2025-03-20 20:50:28,116] Trial 23 finished with value: 0.8022396564483643 and parameters: {'sequence_length': 80, 'hidden_dim': 175, 'num_layers': 2, 'dropout': 0.007423100707065601, 'learning_rate': 0.0008267672586409669, 'batch_size': 8, 'num_epochs': 80, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:50:31,418] Trial 24 pruned. 
[I 2025-03-20 20:51:40,239] Trial 25 finished with value: 0.7693853974342346 and parameters: {'sequence_length': 36, 'hidden_dim': 154, 'num_layers': 1, 'dropout': 0.2070084265747843, 'learning_rate': 0.0014257234563781985, 'batch_size': 16, 'num_epochs': 87, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:51:44,262] Trial 26 pruned. 
[I 2025-03-20 20:51:50,934] Trial 27 pruned. 
[I 2025-03-20 20:51:53,548] Trial 28 pruned. 
[I 2025-03-20 20:51:55,345] Trial 29 pruned. 
[I 2025-03-20 20:51:58,871] Trial 30 pruned. 
[I 2025-03-20 20:53:06,713] Trial 31 finished with value: 0.8262477517127991 and parameters: {'sequence_length': 33, 'hidden_dim': 159, 'num_layers': 1, 'dropout': 0.21814272750012367, 'learning_rate': 0.001503948415731998, 'batch_size': 16, 'num_epochs': 88, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.737079918384552.
[I 2025-03-20 20:53:58,362] Trial 32 finished with value: 0.7314378619194031 and parameters: {'sequence_length': 4, 'hidden_dim': 178, 'num_layers': 1, 'dropout': 0.23380603053231552, 'learning_rate': 0.0010570771786961055, 'batch_size': 16, 'num_epochs': 85, 'optimizer': 'RMSprop'}. Best is trial 32 with value: 0.7314378619194031.
[I 2025-03-20 20:54:02,024] Trial 33 pruned. 
[I 2025-03-20 20:54:03,157] Trial 34 pruned. 
[I 2025-03-20 20:54:04,728] Trial 35 pruned. 
[I 2025-03-20 20:54:06,274] Trial 36 pruned. 
[I 2025-03-20 20:54:07,740] Trial 37 pruned. 
[I 2025-03-20 20:54:11,141] Trial 38 pruned. 
[I 2025-03-20 20:54:13,721] Trial 39 pruned. 
[I 2025-03-20 20:54:15,755] Trial 40 pruned. 
[I 2025-03-20 20:55:20,329] Trial 41 finished with value: 0.7441690564155579 and parameters: {'sequence_length': 29, 'hidden_dim': 161, 'num_layers': 1, 'dropout': 0.1877622038659314, 'learning_rate': 0.0010887960158424278, 'batch_size': 16, 'num_epochs': 87, 'optimizer': 'RMSprop'}. Best is trial 32 with value: 0.7314378619194031.
[I 2025-03-20 20:55:23,229] Trial 42 pruned. 
[I 2025-03-20 20:56:24,333] Trial 43 finished with value: 0.7675689458847046 and parameters: {'sequence_length': 12, 'hidden_dim': 178, 'num_layers': 1, 'dropout': 0.1802991101020573, 'learning_rate': 0.000990289091537514, 'batch_size': 16, 'num_epochs': 95, 'optimizer': 'RMSprop'}. Best is trial 32 with value: 0.7314378619194031.
[I 2025-03-20 20:56:28,421] Trial 44 pruned. 
[I 2025-03-20 20:56:29,740] Trial 45 pruned. 
[I 2025-03-20 20:56:33,723] Trial 46 pruned. 
[I 2025-03-20 20:56:35,405] Trial 47 pruned. 
[I 2025-03-20 20:56:40,337] Trial 48 pruned. 
[I 2025-03-20 20:56:42,941] Trial 49 pruned. 
```

</details>

```
Study statistics: 
 Number of finished trials:  50
  Number of pruned trials:  35
  Number of complete trials:  15
Best trial:
  Value:  0.7314378619194031
  Params: 
    sequence_length: 4
    hidden_dim: 178
    num_layers: 1
    dropout: 0.23380603053231552
    learning_rate: 0.0010570771786961055
    batch_size: 16
    num_epochs: 85
    optimizer: RMSprop
```

In [ ]:
optimization_history = plot_optimization_history(study)
optimization_history.write_image('optimization_history.png')

intermediate_values = plot_intermediate_values(study)
intermediate_values.write_image('intermediate_values.png')

parallel_coordinates = plot_parallel_coordinate(study)
parallel_coordinates.write_image('parallel_coordinates.png')

contour = plot_contour(study)
contour.write_image('contour.png')

slice_ = plot_slice(study)
slice_.write_image('slice.png')

![Optimisation History Plot](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/optimization_history.png)

The validation loss is shown as the objective value in the above optimisation history plot, with the lowest value being for study #32.
Optimisation history shows the validation loss as the objective value, the lowest bein

![Intermediate Values Plot](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/intermediate_values.png)

The intermediate values plot show the average validation loss per study as reported during the trial, which seems to plateau after around 40 epochs.

![Parallel Coordinates Plot](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/parallel_coordinates.png)

The parallel coordinates plot is helpful to see correlations between the different hyperparameters and the objective function. The dark lines for lower objective value scores indicate the better trials, for example there is a strong correlation between a lower mean squared error loss and a batch size of 16.

![Contour Plot](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/contour.png)

The contour plot visualises the relationship between hyperparameters by projecting it onto a 2D plane. For example, it suggests that a model with around 150 latent dimensions and a learning rate of 0.01 produces lower objective function values.

![Slice Plot](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/slice.png)

The slice plot shows the objective function values for a single hyperparameter across all trials, for example RMSProp seems to perform better as an optimizer compared to SGD or Adam.

# Train Model with Best Hyperparameters

This section uses the best parameters found via Optuna trials in the previous section to train a model from scratch over the entire dataset. The best parameters are:
```json
{
    "sequence_length": 4,
    "hidden_dim": 178,
    "num_layers": 1,
    "dropout": 0.23380603053231552,
    "learning_rate": 0.0010570771786961055,
    "batch_size": 16,
    "num_epochs": 85,
    "optimizer": "RMSprop"
}
```

In [ ]:
try:
    best_params = trial.params
except:
    with open('best_parameters.json', 'r') as infile:
        best_params = json.load(infile)
print('Using parameters:', best_params)

dataset = StockDataset(
    csv_file_path=TRAINING_DATA_FILEPATH,
    in_seq_length=best_params['sequence_length'],
    is_train=True,
    train_val_split=0)
data = DataLoader(
    dataset,
    batch_size=best_params['batch_size'],
    shuffle=True)
model = StockLSTM(
    input_dim=N_COMPANIES,
    hidden_dim=best_params['hidden_dim'],
    num_layers=best_params['num_layers'],
    dropout=best_params['dropout']
).to(DEVICE)

criterion = nn.MSELoss()
optimizer = getattr(optim, best_params['optimizer'])(
    model.parameters(),
    lr=best_params['learning_rate'])
train_losses = []
for epoch in range(best_params['num_epochs']):
    model.train()
    total_loss = 0
    for batch in data:
        X = batch['X'].to(DEVICE)
        y = batch['y'].to(DEVICE)

        optimizer.zero_grad()
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(data)
    if (epoch+1) % 10 == 0:
        print(f"Epoch {epoch+1}/{best_params['num_epochs']}, Loss: {avg_loss:.6f}")
    train_losses.append(avg_loss)

torch.save(model.state_dict(), BEST_MODEL_FILEPATH)

plt.figure(figsize=(10, 6))
plt.plot(range(1, best_params['num_epochs']+1), train_losses, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('training_loss.png')
plt.show()

<details open>
  <summary>Training Loss Epochs</summary>

  ```
    Epoch 10/85, Loss: 0.667330
    Epoch 20/85, Loss: 0.574235
    Epoch 30/85, Loss: 0.536828
    Epoch 40/85, Loss: 0.509404
    Epoch 50/85, Loss: 0.480500
    Epoch 60/85, Loss: 0.462396
    Epoch 70/85, Loss: 0.449508
    Epoch 80/85, Loss: 0.442076
  ```
</details>

![Training Loss Plot](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/training_loss.png)

# Make Predictions

This section uses the trained model to make the predictions for 1st April using the previous `in_seq_length` days. The `StandardScaler` instances used for normalising is used to denormalise the predicted output.

In [ ]:
seq_length = dataset.in_seq_length
last_sequence = dataset.normalized_data[:, -seq_length:]
input_tensor = torch.FloatTensor(last_sequence.T).unsqueeze(0).to(DEVICE)

model.eval()
with torch.no_grad():
    prediction_norm = model(input_tensor).squeeze(0).cpu().numpy()
predictions = prediction_norm
for idx in range(len(predictions)):
    predictions[idx] = dataset.scalers[idx].inverse_transform(predictions[idx].reshape(1, -1))
submission = pd.DataFrame({
    'ID': [f'company_{i}' for i in range(N_COMPANIES)],
    'value': predictions.flatten()
})
submission.to_csv('submissions.csv',index=False)

In [ ]:
# use the output directly for Kaggle submissions
file = open('submissions.csv', 'r')
data = file.readlines()
file.close()
print(data)

# Interpretability

This section uses the `Captum` library to understand the LSTM model similar to Lab 6. Since this is a time series prediction of multiple companies:
* Attempt to understand which days from the lookback number of days are most attributed to predict the next day's output.
* Attempt to understand how the output for a particular company attributes to other companies.



In [ ]:
try:
    best_params = trial.params
except:
    with open('best_parameters.json', 'r') as infile:
        best_params = json.load(infile)
print('Using parameters:', best_params)
model = StockLSTM(
    input_dim=N_COMPANIES,
    hidden_dim=best_params['hidden_dim'],
    num_layers=best_params['num_layers'],
    dropout=best_params['dropout']
).to(DEVICE)
model.load_state_dict(torch.load(BEST_MODEL_FILEPATH, weights_only=False))
print(model)

In [ ]:
TARGET_COMPANY = 18  # random company to generate attributions for
TOP_K = 5

model.eval()
train_dataset = StockDataset(
    csv_file_path=TRAINING_DATA_FILEPATH,
    in_seq_length=best_params['sequence_length'],
    is_train=True, 
    train_val_split=VAL_SPLIT
)
validation_dataset = StockDataset(
    csv_file_path=TRAINING_DATA_FILEPATH,
    in_seq_length=best_params['sequence_length'],
    is_train=False, 
    train_val_split=VAL_SPLIT,
    scalers=train_dataset.scalers
)
sample_data = validation_dataset[0]
sample_X = sample_data['X']  # (seq_length, 442)
sample_input = sample_X.unsqueeze(0).to(DEVICE)
sample_input.requires_grad = True

ig = IntegratedGradients(model)
attributions = ig.attribute(
    inputs=sample_input,
    baselines=torch.zeros_like(sample_input).to(DEVICE), 
    target=TARGET_COMPANY,
)
attributions = attributions.cpu().detach().numpy().squeeze(0)  # (seq_length, 442)

In [ ]:
# visualise temporal attributions (which time steps matter most)
plt.figure(figsize=(10, 6))
time_importance = np.abs(attributions).sum(axis=1)  # sum over companies
plt.bar(range(best_params['sequence_length']), time_importance)
plt.xlabel('Time Step')
plt.ylabel('Attribution Magnitude')
plt.title(f'Time Step Importance for Predicting Company {TARGET_COMPANY}')
plt.xticks(range(best_params['sequence_length']))
plt.tight_layout()
plt.savefig('time_step_attribution.png')
plt.show()

![Time Step Attribution](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/time_step_attribution.png)

Since the best parameters use four days as the lookback period to predict the output, the plot shows the importance of each of those 4 days. It suggests that the last (i.e. latest) day has the most influence on the next day's output, with decreasing influence over more previous days. The third-last and fourth-last days have almost similar influence. This model hence prioritises shorter time periods over longer term trends.

In [ ]:
# visualise which companies have the most influence
plt.figure(figsize=(12, 8))
company_importance = np.abs(attributions).mean(axis=0)
top_indices = np.argsort(company_importance)[-TOP_K:]
plt.barh(range(TOP_K), company_importance[top_indices])
plt.yticks(range(TOP_K), top_indices)
plt.xlabel('Attribution Magnitude')
plt.ylabel('Company Index')
plt.title(f'Top {TOP_K} Influential Companies for Predicting Company {TARGET_COMPANY}')
plt.tight_layout()
plt.savefig('company_attribution.png')
plt.show()

![Company Attribution](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/company_attribution.png)

The plot aboves show how other nodes (companies) influence the output for a particular node (company). In this case, the most influential company to predict company 18's stock movement is company 18 itself. Other companies in the top 5 have similar attribution, but lesser than the target company itself. It could suggest that other companies perhaps belong to the same market sector.

In [ ]:
# visualise attribution patterns over time for top influential companies
plt.figure(figsize=(12, 8))
top_indices = np.argsort(company_importance)[-TOP_K:]

for i, idx in enumerate(top_indices):
    plt.plot(range(best_params['sequence_length']),
             attributions[:, idx], 
             label=f'Company {idx}',
             marker='o',
             linewidth=2)

plt.xlabel('Time Step')
plt.ylabel('Attribution Value')
plt.title(f'Attribution Patterns Over Time for Top {TOP_K} Influential Companies')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('company_influence.png')
plt.show()

![Company Influence](https://raw.githubusercontent.com/siddydutta/LSTM-Stock-Market-Prediction/refs/heads/master/company_influence.png)

The final plot shows each of the above five companies contribute numerically to the output for company 18. Interestingly, there is a divergence where two companies suggest a positive stock movement but three companies suggest a negative stock movement. The same company 18 has the strongest negative stock movement resulting in a final output of `-0.032038778`.

# Kaggle Submission (On Public Leaderboard)

The model above achieved a score of `2.52854` on the public test dataset and was submitted under the team name: `Calls on $LSTM`.